In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.metrics import balanced_accuracy_score

In [2]:
test=pd.read_csv("job_change_test.csv")
test_ids = test["id"]
train=pd.read_csv("job_change_train.csv")
df_best_params=pd.read_csv('best_params.csv')

In [3]:

dict_best_params=df_best_params['Best Results'].iloc[0]
type(dict_best_params)
import ast

dict_best_params = ast.literal_eval(dict_best_params)
dict_best_params = {k.replace("clf__", ""): v for k, v in dict_best_params.items()}




In [4]:
categorical_features = [
    "gender", "education", "field_of_studies", "is_studying",
    "county", "is_certified", "size_of_company", "type_of_company","years_since_job_change","years_of_experience"
]

numeric_features = [
    "age", "relative_wage", 
     "hours_of_training"
]


In [5]:
preprocessor=ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features),
                ('nom', StandardScaler(), numeric_features)]
) 

In [6]:
dtc=DecisionTreeClassifier(random_state=42, **dict_best_params)

In [7]:
dt=Pipeline(
    [
        ("Column proessing",preprocessor), 
        ("Decision Tree",dtc)
    ]
     )

In [8]:
X=train.drop(["willing_to_change_job",'id'], axis=1)
y=train['willing_to_change_job']

In [9]:
dt.fit(X,y)
print(balanced_accuracy_score(dt.predict(X),y))


0.7673419604302256


In [10]:
#giving the test
X_test=test.drop('id', axis=1)
prediction=dt.predict(X_test)
df_prediction=pd.DataFrame(prediction, columns=['Prediction'])



In [11]:
result=pd.concat([test_ids, df_prediction], axis=1)

In [12]:
result
result.to_csv("results.csv",index=False)